In [4]:
import os
import torch
import numpy as np
import gensim
import gensim.downloader
from gensim.models import Word2Vec
import jieba
from transformers import AutoTokenizer, AutoModelForMaskedLM
from utilz import *

In [5]:
video_ids, clip_ids, texts, annotations, modes = load_data('../SIMS/label.csv')

In [6]:
texts[:5]

['我不想嫁给李茶',
 '你这是嫁入豪门啊！',
 '我不想嫁入什么豪门，我们不就是豪门吗？',
 '有那么明显吗？',
 '我在这消费这么多钱，我低血糖吃块糖还不行了']

In [7]:
example_seg = list(jieba.lcut(texts[0]))
print(example_seg)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.380 seconds.
Prefix dict has been built successfully.


['我', '不想', '嫁给', '李茶']


# Construct a vocabulary and train a word2vec

In [8]:
vocabs = []
max_len = 0
for s in texts:
    tokens = list(jieba.lcut(s))
    vocabs.append(tokens)
    if len(tokens) > max_len:
        max_len = len(tokens)
    
print(len(vocabs))
print(max_len)

2281
36


In [9]:
vocabs[:10]

[['我', '不想', '嫁给', '李茶'],
 ['你', '这', '是', '嫁入', '豪门', '啊', '！'],
 ['我', '不想', '嫁入', '什么', '豪门', '，', '我们', '不', '就是', '豪门', '吗', '？'],
 ['有', '那么', '明显', '吗', '？'],
 ['我', '在', '这', '消费', '这么', '多钱', '，', '我', '低血糖', '吃块', '糖', '还', '不行', '了'],
 ['够',
  '了',
  '，',
  '每次',
  '都',
  '是',
  '这',
  '一套',
  '，',
  '我',
  '不想',
  '再',
  '听',
  '了',
  '。'],
 ['我',
  '刚才',
  '信号',
  '不好',
  '，',
  '现在',
  '可以',
  '了',
  '，',
  '怎么样',
  '，',
  '妈',
  '，',
  '公司',
  '给',
  '我',
  '安排',
  '的',
  '海景',
  '别墅',
  '楼',
  '。'],
 ['我', '刚才', '是', '一', '害怕', '我', '就', '跟', '王安迪', '撒', '了', '个', '谎'],
 ['说',
  '到',
  '演讲',
  '啊',
  '，',
  '我',
  '不禁',
  '想起',
  '了',
  '我',
  '在',
  '我',
  '的',
  '母校',
  '。'],
 ['经理', '经理', '，', '别忘了', '我', '那', '五十万', '和', '副科长', '。']]

In [10]:
model_own = Word2Vec(sentences=vocabs, vector_size=100, sg=1, min_count=1)
model_own.save("word2vec.model")
model_own.train(vocabs, total_examples=24162, epochs=10)

(168624, 241620)

In [11]:
word_embs = {'train':[], 'valid':[], 'test':[]}

for s, mode in zip(vocabs, modes):
    tmp_embs = []
    for w in s:
        tmp_embs.append(model_own.wv[w])
    if len(tmp_embs)<max_len:
        tmp_embs = np.concatenate([tmp_embs, np.zeros((max_len-len(tmp_embs), 100))], axis=0)
    word_embs[mode].append(tmp_embs[:max_len])
    
save_features(word_embs, './data/textual_wav2vec.pkl')

In [12]:
np.shape(word_embs['train'])

(1368, 36, 100)

In [13]:
gensim.downloader.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [14]:
glove_300 = gensim.downloader.load('glove-wiki-gigaword-300')

[===-----------------------------------------------] 6.2% 23.4/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[========------------------------------------------] 17.3% 65.0/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=============-------------------------------------] 26.3% 99.1/376.1MB downloaded

...

[=======================---------------------------] 47.7% 179.4/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[============================----------------------] 56.2% 211.4/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 376.1/376.1MB downloaded


In [15]:
glove_300.similarity('friend', 'enemy')

0.2220246

# Word Embedding with BERT

In [16]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese", output_hidden_states = True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
word_embs = {'train':[], 'valid':[], 'test':[]}

for text, mode in zip(texts, modes):
    # print(text)
    marked_text = "[CLS] " + text + " [SEP]" 
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        tmp_embs = outputs[-1][0].cpu().detach().numpy()[0] #（N，768）
    if len(tmp_embs) < max_len:
        tmp_embs = np.concatenate([tmp_embs, np.zeros((max_len-len(tmp_embs), 768))], axis=0)
    word_embs[mode].append(tmp_embs[:max_len])
    
save_features(word_embs, './data/textual_bert.pkl')

In [24]:
np.shape(word_embs['train'])

(1368, 36, 768)